#1 Introducción

El siguiente cuadernos realiza la busqueda de un número entero para la version secuencial utilizando solo procesador CPU. En la versión que se utiliza paralelismo utilizando la GPU  también se realiza una busqueda de un número flotante.

El objetivo es comparar los tiempos de ejecución de ambas versiones del algoritmo de busqueda.

#2 Armado del ambiente

Instala en el cuaderno el módulo CUDA de Python. Esto debe ejecutarse solamente para la version que utiliza paralelismo usando la GPU


In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 20.6MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 81kB 12.6MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621010 sha256=943b9efd84583f3c15acba956d60a5b589fde5019c76f30125fdf4cd50dd0df1
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=c9bf0e89a0301f975c9ae34e7d865a087c6f6ceed3f4f9d89e8fccbeb3c98f96
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

Ejecuta el Código CPU secuencial

In [ ]:
from  datetime import datetime

numeroEncontrado = 0

vectorCpu = [1,2,3,4,5]
print("vector con numeros:")
print(vectorCpu)



print("Ingrese un numero a buscar:")
numeroBuscado = input()


for i in range(0,5):
  if vectorCpu[i] == int(numeroBuscado):
    numeroEncontrado=1

    

if numeroEncontrado == 1:
  print("El numero ha sido encontrado")
else:
    print("El numero no ha sido encontrado")


vector con numeros:
[1, 2, 3, 4, 5]
Ingrese un numero a buscar:
2
El numero ha sido encontrado


Ejecuta el Código CPU - GPU aplicando paralelismo

In [ ]:
#@title Parametros { display-mode: "form" }
numeroElementos =  0#@param {type:"number"}
numeroBuscado = 0 #@param {type:"number"}
import pycuda.driver as cuda
import pycuda.autoinit
from   pycuda.compiler import SourceModule

from  datetime import datetime
import numpy



# CPU - Defino la memoria del vector en cpu.
vectorCpu = numpy.random.randn(numeroElementos)
vectorCpu = vectorCpu.astype(numpy.float32())
print("vector con numeros:")
print(vectorCpu)

# CPU - reservo la memoria GPU.
vectorGpu = cuda.mem_alloc(vectorCpu.nbytes)

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod(vectorGpu,vectorCpu)

# CPU - ingreso el numero a buscar
print("Ingrese un numero a buscar:")
numeroBuscado = input()

#CPU - Defino la funcion kernel que ejecutará en GPU
module = SourceModule("""
__global__ void buscarNumero(int n,float numeroBuscado ,float *pvector1)
{
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    
    

    if(idx<n)
    {
      if(pvector1[idx] == numeroBuscado)
      {
          pvector1[0] = 1;
      }
    }
}

""")

kernel = module.get_function("buscarNumero")

dim_hilo = 256
dim_bloque = numpy.int( (n+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

kernel( numpy.int32(n),numpy.float32(numeroBuscado), vectorGpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

cuda.memcpy_dtoh(vectorCpu,vectorGpu)

print("vector con numeros final:")
print(vectorCpu)

if vectorCpu[0]== 1:
  print("El numero ha sido encontrado")
else:
    print("El numero no ha sido encontrado")

#4 Tabla de pasos de ejecución del programa



 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectoes A, B y R.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.



#5 Conclusiones

#6 Bibliografia